# Chapter 6: Seismic Wave Propagation

## 6.1 Seismic Wave Propagation in Two Dimensions

**The scripts referenced in this section are: example08a.py**

We will now expand upon the previous chapter by introducing a vector form of the wave equation. This means
that the waves will have not only a scalar magnitude as for the pressure wave solution, but also a direction. This type of scenario is apparent in wave types that exhibit compressional and transverse particle motion. An example of this would be seismic waves. Wave propagation in the earth can be described by the elastic wave equation:

\begin{equation}
  \rho \frac{\partial^{2}u_{i}}{\partial t^2} - \frac{\partial \sigma_{ij}}{\partial x_{j}} = 0
  \tag{6.1}
\end{equation}

where $\sigma$ is the stress given by

\begin{equation}
  \sigma _{ij} = \lambda u_{k,k} \delta_{ij} + \mu (u_{i,j} + u_{j,i})
  \tag{6.2}
\end{equation}

and $\lambda$ and $\mu$ represent Lame's parameters. Specifically for seismic waves, $\mu$ is the propagation materials shear modulus. In a similar process to the previous chapter, we will use the acceleration solution to solve this PDE. By substituting $a$ directly for $\frac{\partial^{2}u_{i}}{\partial t^2}$ we can derive the
acceleration solution. Using $a$ we can see that Equation (6.1) becomes

\begin{equation} 
  \rho a_{i} - \frac{\partial\sigma_{ij}}{\partial x_{j}} = 0
  \tag{6.3}
\end{equation}

Thus the problem will be solved for acceleration and then converted to displacement using the backwards difference approximation as for the acoustic example in the previous chapter.

Consider now the stress $\sigma$. One can see that the stress consists of two distinct terms:

\begin{equation}
  \lambda u_{k,k} \delta_{ij}
  \tag{6.4a}
\end{equation}

\begin{equation}
  \mu (u_{i,j} + u_{j,i})
  \tag{6.4b}
\end{equation}

One simply recognizes in Equation (6.4a) that $u_{k,k}$ is the trace of the displacement solution and that $\delta_{ij}$ is the kronecker delta function with dimensions equivalent to $u$. The second term Equation (6.4b) is the sum of $u$ with its own transpose. Putting these facts together we see that the spatial differential of the stress is given by the gradient of $u$ and the aforementioned operations. This value is then submitted
to the *escript* PDE as $X$.

To solve wave propagation in *escript*, as usual we need to set up a computation domain first:

In [ ]:
from esys.escript import *
from esys.finley import Rectangle

#Geometric property related variables.
mx = 1000. # model lenght
my = -1000. # model width
ndx = 500 # steps in x direction 
ndy = 500 # steps in y direction
xstep=mx/ndx # calculate the size of delta x
ystep=abs(my/ndy) # calculate the size of delta y

domain=Rectangle(l0=mx,l1=my,n0=ndx, n1=ndy) # create the domain
x=domain.getX() # get the locations of the nodes in the domain

print("Done")

Then we set up a PDE:

In [ ]:
from esys.escript.linearPDEs import LinearPDE, SolverOptions

#Material property related variables.
lam=3.462e9 #lames constant
mu=3.462e9  #bulk modulus
rho=1154.   #density

mypde=LinearPDE(domain) # create pde
mypde.setSymmetryOn() # turn symmetry on
# turn lumping on for more efficient solving
mypde.getSolverOptions().setSolverMethod(SolverOptions.HRZ_LUMPING)
kmat = kronecker(domain) # create the kronecker delta function of the domain
mypde.setValue(D=kmat*rho) #set the general form value D

print("Done")

We also need to define a source:

In [ ]:
import numpy as np

U0=0.01 # amplitude of point source
# will introduce a spherical source at middle left of bottom face
xc=[mx/2,0]
# define small radius around point xc
src_length = 20; print("src_length = ",src_length)
# set initial values for first two time steps with source terms
y=U0*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
src_dir=np.array([0.,-1.]) # defines direction of point source as down
y=y*src_dir
mypde.setValue(y=y) #set the source as a function on the boundary
# initial value of displacement at point source is constant (U0=0.01)
# for first two time steps
u=[0.0,0.0]*whereNegative(x)
u_m1=u

print("Done")

Let us define some time related variables and the save path before we entre the loop:

In [ ]:
tend=0.5    # end time
h=0.0005     # time step
# data recording times
rtime=0.0 # first time to record
rtime_inc=tend/20.0 # time increment to record
#Check to make sure number of time steps is not too large.
print("Time step size=",h, "Expected iterations=",tend/h, " Expected outputs=", tend/rtime_inc)

# where to save output data
savepath = "data/example08a"
mkDir(savepath)

print("Done")

Finally, we start the iteration:

In [ ]:
from esys.weipa import saveVTK
import sys
import os
import time

n=0 # iteration counter
t=0 # time counter

start_time = time.time()
print("Iteration started, please wait...")
while t<tend:
    # get current stress
    g=grad(u); stress=lam*trace(g)*kmat+mu*(g+transpose(g))
    mypde.setValue(X=-stress) # set PDE values
    accel = mypde.getSolution() #get PDE solution for accelleration
    u_p1=(2.*u-u_m1)+h*h*accel #calculate displacement
    u_m1=u; u=u_p1 # shift values by 1
    # save current displacement, acceleration and pressure
    if (t >= rtime):
        saveVTK(os.path.join(savepath,"ex08a.%05d.vtu"%n),displacement=length(u),\
                             acceleration=length(accel),tensor=stress)
        rtime=rtime+rtime_inc #increment data save time
    # increment loop values
    t=t+h; n=n+1
    print("time step %d, t=%s"%(n,t))
    
end_time = time.time()
time = end_time - start_time
print("Simulation completed! Time taken: %s seconds." % time)    

Check the saved vtu files and your results should look like Figure (6.1):

<br>
 <figure>
  <img src="figures/ex08pw.png" width="600">
  <figcaption>
    <center>
      Figure 6.1: Results of Example 08 at various times.
    </center>
  </figcaption>
 </figure>
<br>

## 6.2 Time variant source

**The scripts referenced in this section are: example08b.py**

The previous examples use impulsive sources which are smooth in space but not time. It is however, advantageous to have a time smoothed source as it can reduce the temporal frequency range and thus mitigate aliasing in the solution.

It is quite simple to implement a source which is smooth in time. In addition to the original source function
the only extra requirement is a time function. For this example the time variant source will be the derivative of a Gaussian curve defined by the required dominant frequency:

In [ ]:
# makes an interactive plotting window
%matplotlib notebook

import pylab as pl

# where to save output data
savepath = "data/example08b"
mkDir(savepath)

h=0.0005    # time step

#Creating the time function of the source.
dfeq=50 #Dominant Frequency
a = 2.0 * (np.pi * dfeq)**2.0
t0 = 5.0 / (2.0 * np.pi * dfeq)
srclength = 5. * t0
ls = int(srclength/h)
print('source length',ls)

source=np.zeros(ls,'float') # source array
time=np.zeros(ls,'float')   # time values
ampmax=0
for it in range(0,ls):
    t = it*h
    tt = t-t0
    dum1 = np.exp(-a * tt * tt)
    source[it] = -2. * a * tt * dum1
    if (abs(source[it]) > ampmax):
        ampmax = abs(source[it])
    time[it]=t*h
   
pl.clf(); 
pl.plot(source)
pl.savefig(os.path.join(savepath,'source.png'))

print("Done")

We then build the source and the first two time steps via:

In [ ]:
mx = 1000. # model lenght
my = 1000. # model width
ndx = 300 # steps in x direction 
ndy = 300 # steps in y direction
xstep=mx/ndx # calculate the size of delta x
ystep=abs(my/ndy) # calculate the size of delta y

domain=Rectangle(l0=mx,l1=my,n0=ndx, n1=ndy) # create the domain
x=domain.getX() # get the locations of the nodes in the domani

# define small radius around point xc
src_length = 40; print("src_length = ",src_length)
xc=[mx/2,0]
# set initial values for first two time steps with source terms
y=source[0]*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
src_dir=np.array([0.,1.]) # defines direction of point source as down
y=y*src_dir

mypde=LinearPDE(domain) # create pde
mypde.setSymmetryOn() # turn symmetry on
# turn lumping on for more efficient solving
mypde.getSolverOptions().setSolverMethod(SolverOptions.HRZ_LUMPING)
kmat = kronecker(domain) # create the kronecker delta function of the domain
mypde.setValue(D=kmat*rho) #set the general form value D
mypde.setValue(y=y) #set the source as a function on the boundary
# for first two time steps
u=[0.0,0.0]*wherePositive(x)
u_m1=u

print("Done")

Finally, for the length of the source, we are required to update each new solution in the iterative section of the solver:

In [ ]:
import time

rtime=0.0 # first time to record
n=0 # iteration counter
t=0 # time counter

start_time = time.time()
print("Iteration started, please wait...")
while t<tend:
    # get current stress
    g=grad(u); stress=lam*trace(g)*kmat+mu*(g+transpose(g))
    mypde.setValue(X=-stress) # set PDE values
    accel = mypde.getSolution() #get PDE solution for accelleration
    u_p1=(2.*u-u_m1)+h*h*accel #calculate displacement
    u_m1=u; u=u_p1 # shift values by 1
    # save current displacement, acceleration and pressure
    if (t >= rtime):
        saveVTK(os.path.join(savepath,"ex08b.%05d.vtu"%n),displacement=length(u),\
                             acceleration=length(accel),tensor=stress)
        rtime=rtime+rtime_inc #increment data save time
    # increment loop values
    t=t+h; n=n+1
    if (n < ls):
        y=source[n]*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
        y=y*src_dir; mypde.setValue(y=y) #set the source as a function on the boundary
    print("time step %d, t=%s"%(n,t))

end_time = time.time()
time = end_time - start_time
print("Simulation completed! Time taken: %s seconds." % time)

## 6.3 Absorbing Boundary Conditions

To mitigate the effect of the boundary on the model, absorbing boundary conditions can be introduced. These con-
ditions effectively dampen the wave energy as they approach the boundary and thus prevent that energy from being
reflected. This type of approach is typically used when a model is shrunk to decrease computational requirements. In practise this applies to almost all models, especially in earth sciences where the entire planet or a large enough portional of it cannot be modelled efficiently when considering small scale problems. It is impractical to calculate the solution for an infinite model and thus ABCs allow us the create an approximate solution with small to zero boundary effects on a model with a solvable size.

To dampen the waves, the method of Cerjan (1985) where the solution and the stress are multiplied by a damping function defined on $n$ nodes of the domain adjacent to the boundary, given by:

\begin{equation}
  \gamma =\sqrt{\frac{| -log( \gamma _{b} ) |}{n^2}}
  \tag{6.5}
\end{equation}

\begin{equation}
  y=e^{-(\gamma x)^2}
  \tag{6.6}
\end{equation}

This is applied to the bounding 20-50 pts of the model using the location specifiers of *escript*:

In [ ]:
# Define where the boundary decay will be applied.
bn=50.
bleft=xstep*bn; bright=mx-(xstep*bn); bbot=my-(ystep*bn)
# btop=ystep*bn # don't apply to force boundary!!!

# locate these points in the domain
left=x[0]-bleft; right=x[0]-bright; bottom=x[1]-bbot

tgamma=0.85   # decay value for exponential function
def calc_gamma(G,npts):
    func=np.sqrt(abs(-1.*np.log(G)/(npts**2.)))
    return func

gleft  = calc_gamma(tgamma,bleft)
gright = calc_gamma(tgamma,bleft)
gbottom= calc_gamma(tgamma,ystep*bn)

print('gamma', gleft,gright,gbottom)

# calculate decay functions
def abc_bfunc(gamma,loc,x,G):
    func=exp(-1.*(gamma*abs(loc-x))**2.)
    return func

fleft=abc_bfunc(gleft,bleft,x[0],tgamma)
fright=abc_bfunc(gright,bright,x[0],tgamma)
fbottom=abc_bfunc(gbottom,bbot,x[1],tgamma)
# apply these functions only where relevant
abcleft=fleft*whereNegative(left)
abcright=fright*wherePositive(right)
abcbottom=fbottom*wherePositive(bottom)
# make sure the inside of the abc is value 1
abcleft=abcleft+whereZero(abcleft)
abcright=abcright+whereZero(abcright)
abcbottom=abcbottom+whereZero(abcbottom)
# multiply the conditions together to get a smooth result
abc=abcleft*abcright*abcbottom

#visualise the boundary function
abcT=abc.toListOfTuples()
abcT=np.reshape(abcT,(ndx+1,ndy+1))
pl.figure(2)
pl.clf(); pl.imshow(abcT); pl.colorbar(); 
pl.savefig(os.path.join(savepath,"abc.png"))

print("Done")

Note that the boundary conditions are not applied to the surface, as this is effectively a free surface where normal reflections would be experienced. Special conditions can be introduced at this surface if they are known. 

Now, after resetting the initial vairalbes and the PDE, we can solve iteratively with the absorbing boundary condition:

In [ ]:
import time

mypde=LinearPDE(domain) # create pde
mypde.setSymmetryOn() # turn symmetry on
# turn lumping on for more efficient solving
mypde.getSolverOptions().setSolverMethod(SolverOptions.HRZ_LUMPING)
kmat = kronecker(domain) # create the kronecker delta function of the domain
mypde.setValue(D=kmat*rho) #set the general form value D

y=source[0]*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
src_dir=np.array([0.,1.]) # defines direction of point source as down
y=y*src_dir
mypde.setValue(y=y) #set the source as a function on the boundary
# for first two time steps
u=[0.0,0.0]*whereNegative(x)
u_m1=u

rtime=0.0 # first time to record
n=0 # iteration counter
t=0 # time counter

start_time = time.time()
print("Iteration started, please wait...")
while t<tend:
    # get current stress
    g=grad(u); stress=lam*trace(g)*kmat+mu*(g+transpose(g))
    mypde.setValue(X=-stress*abc) # set PDE values
    accel = mypde.getSolution() #get PDE solution for accelleration
    u_p1=(2.*u-u_m1)+h*h*accel #calculate displacement
    u_p1=u_p1*abc       # apply boundary conditions
    u_m1=u; u=u_p1 # shift values by 1
    # save current displacement, acceleration and pressure
    if (t >= rtime):
        saveVTK(os.path.join(savepath,"ex08b_abc.%05d.vtu"%n),displacement=length(u),\
                             acceleration=length(accel),tensor=stress)
        rtime=rtime+rtime_inc #increment data save time
    # increment loop values
    t=t+h; n=n+1
    if (n < ls):
        y=source[n]*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
        y=y*src_dir; mypde.setValue(y=y) #set the source as a function on the boundary
    print("time step %d, t=%s"%(n,t))
    
end_time = time.time()
time = end_time - start_time
print("Simulation completed! Time taken: %s seconds." % time)    

Figure (6.2) comprares the displacement results with and without absorbing boundary conditions at various time steps.

<br>
 <figure>
  <img src="figures/ex08compare_abc.png" width="700">
  <figcaption>
    <center>
      Figure 6.2: Results of Example 08 at various times, with and without absorbing boundary condition.
    </center>
  </figcaption>
 </figure>
<br>

## 6.4 Second order Meshing

For stiff problems like the wave equation it is often prudent to implement second order meshing. This creates a more accurate mesh approximation with some increased processing cost. To turn second order meshing on, the `rectangle` function accepts an `order` keyword argument:

In [ ]:
domain=Rectangle(l0=mx,l1=my,n0=ndx, n1=ndy,order=2) # create the domain

print("Done")

The difference between secord order mesh and first order mesh in shown in Figure (6.3).

<br>
 <figure>
  <img src="figures/ex081st_vs_2nd_meshes.png" width="700">
  <figcaption>
    <center>
      Figure 6.3: First and second order meshes.
    </center>
  </figcaption>
 </figure>
<br>

Note that when implementing second order meshing, a smaller timestep is required than for first order meshes as the second order essentially reduces the size of the mesh by half:

In [ ]:
h=0.0002    # reduced time step

print("Done")

The updated time variant source is:

In [ ]:
#Creating the time function of the source.
ls = int(srclength/h)
print('source length',ls)

source=np.zeros(ls,'float') # source array
time=np.zeros(ls,'float')   # time values
ampmax=0
for it in range(0,ls):
    t = it*h
    tt = t-t0
    dum1 = np.exp(-a * tt * tt)
    source[it] = -2. * a * tt * dum1
    if (abs(source[it]) > ampmax):
        ampmax = abs(source[it])
    time[it]=t*h

pl.figure(3)    
pl.clf() 
pl.plot(source)
pl.savefig(os.path.join(savepath,'source2.png'))

print("Done")

After updating or resetting all other relevant variables, the displacement is solved again on this second order mesh:

In [ ]:
import time

x=domain.getX() # get the locations of the nodes in the domain

left=x[0]-bleft; right=x[0]-bright; bottom=x[1]-bbot

fleft=abc_bfunc(gleft,bleft,x[0],tgamma)
fright=abc_bfunc(gright,bright,x[0],tgamma)
fbottom=abc_bfunc(gbottom,bbot,x[1],tgamma)

abcleft=fleft*whereNegative(left)
abcright=fright*wherePositive(right)
abcbottom=fbottom*wherePositive(bottom)
# make sure the inside of the abc is value 1
abcleft=abcleft+whereZero(abcleft)
abcright=abcright+whereZero(abcright)
abcbottom=abcbottom+whereZero(abcbottom)
# multiply the conditions together to get a smooth result
abc=abcleft*abcright*abcbottom

mypde=LinearPDE(domain) # create pde
mypde.setSymmetryOn() # turn symmetry on
# turn lumping on for more efficient solving
mypde.getSolverOptions().setSolverMethod(SolverOptions.HRZ_LUMPING)
kmat = kronecker(domain) # create the kronecker delta function of the domain
mypde.setValue(D=kmat*rho) #set the general form value D

# define small radius around point xc
src_length = 40; print("src_length = ",src_length)
# set initial values for first two time steps with source terms
y=source[0]*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
src_dir=np.array([0.,1.]) # defines direction of point source as down
y=y*src_dir
mypde.setValue(y=y) #set the source as a function on the boundary
# for first two time steps
u=[0.0,0.0]*wherePositive(x)
u_m1=u

rtime=0.0 # first time to record
n=0 # iteration counter
t=0 # time counter

start_time = time.time()
print("Iteration started, please wait...")
while t<tend:
    # get current stress
    g=grad(u); stress=lam*trace(g)*kmat+mu*(g+transpose(g))
    mypde.setValue(X=-stress*abc) # set PDE values
    accel = mypde.getSolution() #get PDE solution for accelleration
    u_p1=(2.*u-u_m1)+h*h*accel #calculate displacement
    u_p1=u_p1*abc       # apply boundary conditions
    u_m1=u; u=u_p1 # shift values by 1
    # save current displacement, acceleration and pressure
    if (t >= rtime):
        saveVTK(os.path.join(savepath,"ex08b_abc_2nd.%05d.vtu"%n),displacement=length(u),\
                             acceleration=length(accel),tensor=stress)
        rtime=rtime+rtime_inc #increment data save time
    # increment loop values
    t=t+h; n=n+1
    if (n < ls):
        y=source[n]*(cos(length(x-xc)*3.1415/src_length)+1)*whereNegative(length(x-xc)-src_length)
        y=y*src_dir; mypde.setValue(y=y) #set the source as a function on the boundary
    print("time step %d, t=%s"%(n,t))
    
end_time = time.time()
time = end_time - start_time
print("Simulation completed! Time taken: %s seconds." % time)

Figure 6.4 compares the displacement results obtained on secord order and first order meshes at 0.2 seconds. 

<br>
 <figure>
  <img src="figures/ex081st_vs_2nd_displacement.png" width="700">
  <figcaption>
    <center>
      Figure 6.4: Displacement results simulated on first and second order meshes, at t=0.2 sec.
    </center>
  </figcaption>
 </figure>
<br>

## 6.5 Pycad example

**The scripts referenced in this section are: example08c.py**

To make the problem more interesting we will now introduce an interface to the middle of the domain (Figure 6.5). In fact we will use the same domain as we did for different set of material properties on either side of the interface (example 5 in Section 4.4).

<br>
 <figure>
  <img src="figures/gmsh-example08c.png" width="600">
  <figcaption>
    <center>
      Figure 6.5: Domain geometry for example08c.py showing line tangents.
    </center>
  </figcaption>
 </figure>
<br>



## References

C. Cerjan. A nonreflecting boundary condition for discrete acoustic and elastic wave equations. Geophysics,
50, 1985. doi: 10.1190/1.1441945.